## Install necessary packages

### For easy implementation, run this notebook on Google Colab

In [ ]:
%%capture

!pip install flair

## Mount Google Drive and load data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Prepare data
data_path = "/content/drive/MyDrive/"
exp_id = "presto-german-160623-10"
save_dir = "/content/drive/MyDrive/presto-exp-checkpoints/"+exp_id
model_name = 'xlm-roberta-base'
model_name = 'google/muril-base-cased'

lines = open(data_path+"data_de.dis",'r').readlines()
labels = open(data_path+"data_de.labels",'r').readlines()

In [ ]:
print(lines[:5])
print(labels[:5])

['zeige mir angebote zeigen\n', 'pausiere mein mein lauftraining\n', 'meine meine mitfahrgelegenheit absagen\n', 'pause pausiere das laufen\n', 'beende beende meinen workout\n']
['1 0 0 0\n', '0 1 0 0\n', '1 0 0 0\n', '1 0 0 0\n', '1 0 0 0\n']


In [ ]:
lines[0][:-1]

'zeige mir angebote zeigen'

## Prepare files for Train-Valid-Test Split

In [ ]:
import random
temp = list(zip(lines, labels))
random.shuffle(temp)
res1, res2 = zip(*temp)
# res1 and res2 come out as tuples, and so must be converted to lists.
lines, labels = list(res1), list(res2)

train_lines = lines[:int(0.8*len(lines))]
train_labels = labels[:int(0.8*len(labels))]
val_lines = lines[int(0.8*len(lines)):int(0.9*len(lines))]
val_labels = labels[int(0.8*len(labels)):int(0.9*len(labels))]
test_lines = lines[int(0.9*len(lines)):]
test_labels = labels[int(0.9*len(labels)):]

## Data Processing for Token Classification

In [ ]:
import pandas as pd
sentence_col = []
word_col = []
tag_col = []
for i in range(len(train_lines)):
  sent = train_lines[i][:-1]
  label = train_labels[i][:-1]
  words = sent.split(" ")
  tags = label.split(" ")
  sent_temp = ["Sentence: "+str(i+1)]
  for j in range(len(words)-1):
    sent_temp.append(None)
  sentence_col.extend(sent_temp)
  word_col.extend(words)
  tag_col.extend(tags)

df_train = pd.DataFrame()
df_train['Sentence #'] = sentence_col
df_train['Word'] = word_col
df_train['Tag'] = tag_col

sentence_col = []
word_col = []
tag_col = []
for i in range(len(val_lines)):
  sent = val_lines[i][:-1]
  label = val_labels[i][:-1]
  words = sent.split(" ")
  tags = label.split(" ")
  sent_temp = ["Sentence: "+str(i+1)]
  for j in range(len(words)-1):
    sent_temp.append(None)
  sentence_col.extend(sent_temp)
  word_col.extend(words)
  tag_col.extend(tags)

df_val = pd.DataFrame()
df_val['Sentence #'] = sentence_col
df_val['Word'] = word_col
df_val['Tag'] = tag_col

sentence_col = []
word_col = []
tag_col = []
for i in range(len(test_lines)):
  sent = test_lines[i][:-1]
  label = test_labels[i][:-1]
  words = sent.split(" ")
  tags = label.split(" ")
  sent_temp = ["Sentence: "+str(i+1)]
  for j in range(len(words)-1):
    sent_temp.append(None)
  sentence_col.extend(sent_temp)
  word_col.extend(words)
  tag_col.extend(tags)

df_test = pd.DataFrame()
df_test['Sentence #'] = sentence_col
df_test['Word'] = word_col
df_test['Tag'] = tag_col

In [ ]:
df_val[:10]

,Sentence #,Word,Tag
0,Sentence: 1,mhmm,1
1,None,das,0
2,None,taxi,0
3,None,kann,0
4,None,weg,0
5,Sentence: 2,zahle,0
6,None,die,0
7,None,rechnung,0
8,None,mit,1
9,None,äh,1


In [ ]:
import os
os.makedirs("data_dir",exist_ok = True)

In [ ]:
with open("data_dir/train.txt",'w') as write_file:
  for i in range(len(df_train)):
    if(df_train['Sentence #'][i]!=None and i!=0):
      write_file.write("\n")
    write_file.write(df_train['Word'][i]+" "+df_train['Tag'][i]+"\n")


with open("data_dir/val.txt",'w') as write_file:
  for i in range(len(df_val)):
    if(df_val['Sentence #'][i]!=None and i!=0):
      write_file.write("\n")
    write_file.write(df_val['Word'][i]+" "+df_val['Tag'][i]+"\n")


with open("data_dir/test.txt",'w') as write_file:
  for i in range(len(df_test)):
    if(df_test['Sentence #'][i]!=None and i!=0):
      write_file.write("\n")
    write_file.write(df_test['Word'][i]+" "+df_test['Tag'][i]+"\n")


In [ ]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus

# define columns
columns = {0 : 'text', 1 : 'dc'}
# directory where the data resides
data_folder = 'data_dir/'
# initializing the corpus
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file = 'train.txt',
                              test_file = 'test.txt',
                              dev_file = 'val.txt')

2023-06-16 20:30:49,132 Reading data from data_dir
2023-06-16 20:30:49,133 Train: data_dir/train.txt
2023-06-16 20:30:49,134 Dev: data_dir/val.txt
2023-06-16 20:30:49,136 Test: data_dir/test.txt


In [ ]:
print(len(corpus.train))

2472


In [ ]:
# tag to predict
label_type = 'dc'
# make tag dictionary from the corpus
#tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)
#print(tag_dictionary)

label_dict = corpus.make_label_dictionary(label_type=label_type, add_unk=False)
print(label_dict)

2023-06-16 20:30:49,667 Computing label dictionary. Progress:


2472it [00:00, 36168.14it/s]

2023-06-16 20:30:49,778 Dictionary created for label 'dc' with 2 values: 0 (seen 13943 times), 1 (seen 3861 times)
Dictionary with 2 tags: 0, 1


## Define Model

In [ ]:
from flair.embeddings import TransformerWordEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer

embeddings = TransformerWordEmbeddings(model=model_name,
                                       layers="-1",
                                       subtoken_pooling="first",
                                       fine_tune=False,
                                       use_context=True,
                                       )

In [ ]:
from flair.models import SequenceTagger
tagger : SequenceTagger = SequenceTagger(hidden_size=256,
                                       embeddings=embeddings,
                                       tag_dictionary=label_dict,
                                       tag_type=label_type,
                                       use_crf=False,
                                       use_rnn=True)
print(tagger)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(count_parameters(tagger))

#238150676-238150660 = CRF
#240250884 - 238150660 = RNN

2023-06-16 20:32:44,619 SequenceTagger predicts: Dictionary with 4 tags: 0, 1, <START>, <STOP>
SequenceTagger(
  (embeddings): TransformerWordEmbeddings(
    (model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(197286, 768)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output):

## Start Training

In [ ]:
from flair.trainers import ModelTrainer
trainer : ModelTrainer = ModelTrainer(tagger, corpus)
trainer.train(save_dir,
              learning_rate=0.1,
              mini_batch_size=16,
              max_epochs=150)

2023-06-16 13:24:07,827 ----------------------------------------------------------------------------------------------------
2023-06-16 13:24:07,833 Model: "SequenceTagger(
  (embeddings): TransformerWordEmbeddings(
    (model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(197286, 768)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout

100%|██████████| 20/20 [00:02<00:00,  8.42it/s]

2023-06-16 13:24:28,932 Evaluating as a multi-label problem: False


2023-06-16 13:24:28,961 DEV : loss 0.513359546661377 - f1-score (micro avg)  0.7865
2023-06-16 13:24:28,975 BAD EPOCHS (no improvement): 0
2023-06-16 13:24:28,980 saving best model
2023-06-16 13:24:32,524 ----------------------------------------------------------------------------------------------------
2023-06-16 13:24:32,707 epoch 2 - iter 15/155 - loss 0.49744385 - time (sec): 0.18 - samples/sec: 9708.14 - lr: 0.100000
2023-06-16 13:24:32,897 epoch 2 - iter 30/155 - loss 0.50454125 - time (sec): 0.37 - samples/sec: 9332.42 - lr: 0.100000
2023-06-16 13:24:33,083 epoch 2 - iter 45/155 - loss 0.50654265 - time (sec): 0.56 - samples/sec: 9388.00 - lr: 0.100000
2023-06-16 13:24:33,270 epoch 2 - iter 60/155 - loss 0.51345044 - time (sec): 0.74 - samples/sec: 9440.95 - lr: 0.100000
2023-06-16 13:24:33,443 epoch 2 - iter 75/155 - loss 0.51694891 - time (sec): 0.92 - samples/sec: 9508.34 - lr: 0.100000
2023-06-16 13:24:33,617 epoch 2 - iter 90/155 - loss 0.51892696 - time (sec): 1.09 - samp

100%|██████████| 20/20 [00:00<00:00, 132.30it/s]

...

2023-06-16 13:26:54,883 DEV : loss 0.4168577492237091 - f1-score (micro avg)  0.8191
2023-06-16 13:26:54,901 Epoch    60: reducing learning rate of group 0 to 9.7656e-05.
2023-06-16 13:26:54,903 BAD EPOCHS (no improvement): 4
2023-06-16 13:26:54,909 ----------------------------------------------------------------------------------------------------
2023-06-16 13:26:54,910 ----------------------------------------------------------------------------------------------------
2023-06-16 13:26:54,913 learning rate too small - quitting training!
2023-06-16 13:26:54,915 ----------------------------------------------------------------------------------------------------
2023-06-16 13:26:58,735 ----------------------------------------------------------------------------------------------------
2023-06-16 13:27:06,331 SequenceTagger predicts: Dictionary with 2 tags: 0, 1


100%|██████████| 20/20 [00:02<00:00,  8.00it/s]

2023-06-16 13:27:09,223 Evaluating as a multi-label problem: False


2023-06-16 13:27:09,254 0.8107	0.8107	0.8107	0.8107
2023-06-16 13:27:09,257 
Results:
- F-score (micro) 0.8107
- F-score (macro) 0.5482
- Accuracy 0.8107

By class:
              precision    recall  f1-score   support

           0     0.8093    0.9949    0.8926      1762
           1     0.8571    0.1156    0.2038       467

    accuracy                         0.8107      2229
   macro avg     0.8332    0.5553    0.5482      2229
weighted avg     0.8193    0.8107    0.7483      2229

2023-06-16 13:27:09,260 ----------------------------------------------------------------------------------------------------


{'test_score': 0.8106774338268282,
 'dev_score_history': [0.7865219357756671,
  0.7865219357756671,
  0.7865219357756671,
  0.7865219357756671,
  0.7865219357756671,
  0.7865219357756671,
  0.7865219357756671,
  0.7865219357756671,
  0.7865219357756671,
  0.7865219357756671,
  0.7865219357756671,
  0.7919493441881501,
  0.7869742198100407,
  0.8168249660786974,
  0.8091361374943464,
  0.8086838534599728,
  0.8014473089099955,
  0.8086838534599728,
  0.8082315694255993,
  0.8086838534599728,
  0.819538670284939,
  0.8132066938037087,
  0.8186341022161918,
  0.8127544097693351,
  0.8104929895974672,
  0.8154681139755766,
  0.8154681139755766,
  0.8132066938037087,
  0.8141112618724559,
  0.8163726820443238,
  0.8145635459068294,
  0.8163726820443238,
  0.8186341022161918,
  0.8181818181818182,
  0.8186341022161918,
  0.8163726820443238,
  0.8181818181818182,
  0.8186341022161918,
  0.8181818181818182,
  0.8159203980099502,
  0.8159203980099502,
  0.8177295341474446,
  0.8181818181818182,

## Model Inference

In [ ]:
sent1 = "Suche nach ähm geöffneten Elektronikläden."
sent2 = "Statistik meiner meiner Herzfrequenz aufrufen."
sent3 = "Zahle meine Strom meine Stromrechnung."
sent4 = "Gesundheitsdaten zeige mir die Gesundheitsdaten."
sent5 = "Könnten Sie eine Erinnerungsmail machen."
sentences = []
sentences.append(sent1)
sentences.append(sent2)
sentences.append(sent3)
sentences.append(sent4)
sentences.append(sent5)

In [ ]:
from flair.data import Sentence
from flair.models import SequenceTagger
# load the trained model
model = SequenceTagger.load(save_dir+'/best-model.pt')
# create example sentence
for x in sentences:
  sentence = Sentence(x)
  # predict the tags
  model.predict(sentence)
  print(sentence.to_tagged_string())

2023-06-16 13:27:16,914 SequenceTagger predicts: Dictionary with 2 tags: 0, 1
Sentence[6]: "Suche nach ähm geöffneten Elektronikläden." → ["Suche"/0, "nach"/1, "ähm"/1, "geöffneten"/0, "Elektronikläden"/0, "."/0]
Sentence[6]: "Statistik meiner meiner Herzfrequenz aufrufen." → ["Statistik"/0, "meiner"/0, "meiner"/0, "Herzfrequenz"/0, "aufrufen"/0, "."/0]
Sentence[6]: "Zahle meine Strom meine Stromrechnung." → ["Zahle"/0, "meine"/0, "Strom"/0, "meine"/0, "Stromrechnung"/0, "."/0]
Sentence[6]: "Gesundheitsdaten zeige mir die Gesundheitsdaten." → ["Gesundheitsdaten"/0, "zeige"/0, "mir"/0, "die"/0, "Gesundheitsdaten"/0, "."/0]
Sentence[6]: "Könnten Sie eine Erinnerungsmail machen." → ["Könnten"/0, "Sie"/0, "eine"/0, "Erinnerungsmail"/0, "machen"/0, "."/0]
